# Домашнее задание 1. Извлечение ключевых слов

При выполнении домашнего задания можно пользоваться материалами лекций и семинаров.

### Описание задания

1. (1 балл) Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. 
Предполагается, что вы найдете источник текстов, в котором **уже выделены** ключевые слова.
Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.

2. (2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой.
Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).

3. (2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25, ...)

4. (2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun).
Применить эти фильтры к спискам ключевых слов.

4. (2  балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона:
с учётом морфосинтаксических шаблонов и без них.

5. (1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется);
предложить свои методы решения этих проблем.

### Критерии оценки

В каждом пункте указано количество баллов.

### Формат сдачи задания

Jupyter-notebook на гитхабе (запишите адрес своего репозитория [сюда](https://docs.google.com/forms/d/e/1FAIpQLSfvoxiOKm9jnHO6v_ivGOeA3TKBT7Hg7bQlHa56MuALeMIcvQ/viewform?usp=sf_link))

### Дедлайн

9 ноября 2020 23:59мск


---



### 1: (1 балл) Подготовить мини-корпус (не меньше 4 текстов, примерный общий объём - 3-5 тысяч токенов) с разметкой ключевых слов. 

Предполагается, что вы найдете источник текстов, в котором уже выделены ключевые слова. Укажите источник корпуса и опишите, в каком виде там были представлены ключевые слова.


---


Я решила взять тексты из корпуса текстов [этого проекта](https://github.com/avl33/nrlpk/) (нашла вот [тут](https://habr.com/ru/post/468141/) - описание). Он насчитывает 53 текста (я ограничилась количеством  ). Внутри директории каждого текста в проекте лежат следующие штуки:
1. words.csv — список релевантных слов, включая список 
неидентифицированных;
2. **keys.csv** — список ключевых слов, сейчас содержит, кроме маркированных выражений ещё и слова, которые повторяются по тексту не менее заданного числа раз – в данном случае не менее 4 раз; **те данные, которые нам нужны для текста - забираем этот файл для каждого текста и создаем датасет**
3. garbage.csv — список неидентифицированных слов;
4. descr_words.csv — описание (статистика) к списку всех слов текста;
5. descr_keys.csv — описание (статистика) к списку ключевых слов


keys.csv - "таблички", где хранятся сами ключевые слова для текста + разная информация о них (слово, лемма, часть речи и т.д. и количественные и качественные признаки существования слова в тексте/предложении), при этом одно и то же слово повторяется в файле (входит в него столько раз, сколько встречается (см. count_word_text))

Таким образом, корпус состояит из 5 текстов (объем: 3540 слов):

*   Текст [26](https://vpk.name/news/311200_pentagon_dolzhen_perevooruzhitsya.html): 599 слов
*   Текст [31](https://vpk.name/news/312304_vyizov_prinyat_v_rossiiskih_voiskah_poyavitsya_svoya_sotovaya_svyaz.html): 699 слов
*   Текст [27](https://vpk.name/news/311129_nalet_starinyi.html): 858 слов
*   Текст [32](https://vpk.name/news/313811_die_welt_germaniya_mechta_ob_osobom_puti.html): 803 слова
*   Текст [30](https://vpk.name/news/312184_pentagon_ne_proschital_riski_ekspert_o_voine_ssha_i_rossii_na_more.html): 581 слово


In [1]:
from google.colab import drive
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/Вышечка/4 курс/автобрея

/content/gdrive/My Drive/Вышечка/4 курс/автобрея


In [3]:
import pandas as pd

In [4]:
texts = []
keys_start = []
only_keys = []

all_docs_txt = ["text_26.txt", "text_27.txt", "text_30.txt", "text_31.txt", "text_32.txt"]
all_docs_csv = ["keys_26.csv", "keys_27.csv", "keys_30.csv", "keys_31.csv", "keys_32.csv"]

for doc_txt in all_docs_txt:
    with open(doc_txt, "r", encoding='utf-8') as file:
        content = file.read()
        texts.append(content.replace('\n', ' '))

In [5]:
for doc_csv in all_docs_csv:
    d = {}   # словарь для ключевые слов
    k_l = []  # список ключевых слов

    df = pd.read_csv(doc_csv, sep='|')

    # берем только леммы, части речи и их количество в тексте
    
    lem = df['lemm'].tolist()
    pos = df['partlang'].tolist()
    cnt = df['count_word_text'].tolist()
    ind = 0
    for l in lem:
        if l not in d:
            d[l] = [pos[ind], cnt[ind]]
            ind += 1
        else:
            ind += 1

    for k in d.keys(): 
        k_l.append(k)
    
    only_keys.append(k_l)

    keys_start.append(d)

In [6]:
data = {'texts' : texts,
        'key_words' : keys_start,
        'keys': only_keys}

df = pd.DataFrame(data)

In [7]:
# корпус

df

,texts,key_words,keys
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент..."
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет..."
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг..."
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран..."
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г..."


In [8]:
!pip install razdel

In [9]:
from razdel import tokenize

In [10]:
import re

In [11]:
# на всякий случай (вдруг на сайте было неверно) добавим информацию о количестве токенов

def count_w(sentence):
    tokens = list(tokenize(str(sentence)))
    words = [_.text for _ in tokens]
    only_words = []
    for w in words:
        # только токены
        ru = re.findall(r'[А-Яа-я]+', w)
        eng = re.findall(r'[A-Za-z]+', w)
        if len(ru) != 0 or len(eng) != 0:
            only_words.append(w)
    return len(only_words)

In [12]:
df["length"] = df["texts"].apply(count_w)

In [13]:
df

,texts,key_words,keys,length
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г...",825


In [14]:
# посчитаем, уложились ли мы

df.length.sum()

3641

### 2: (2 балла) Разметить ключевые слова самостоятельно. Оценить пересечение с имеющейся разметкой. Составить эталон разметки (например, пересечение или объединение вашей разметки и исходной).

Слова размечать будем путем прочтения текстов и выделения из них ключевых слов. После - сравним.

In [15]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords

In [17]:
from nltk.probability import FreqDist

In [18]:
!pip install --quiet pymorphy2[fast]

In [19]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [20]:
ru_stops = set(stopwords.words('russian'))

В изначальном проекте учитывали слова, которые встречаются больше, чем 4 раза в тексте, поэтому просто посмотри на самые частотные слова (15 штук) и подумаем об этом, когда будем выделять ключевые

In [21]:
def count_top_words(sent):
    tokens = list(tokenize(str(sent)))
    words = [_.text for _ in tokens]
    only_words = []
    # получаем чистые лемматизированные тексты
    for w in words:
        ru = re.findall(r'[А-Яа-я]+', w)
        eng = re.findall(r'[A-Za-z]+', w)
        if len(ru) != 0 or len(eng) != 0:
            wrd = morph.parse(w)[0].normal_form
            if wrd not in ru_stops:
                only_words.append(wrd)
    snt = ' '.join([e for e in only_words])
    fdist = FreqDist(snt.split(' '))
    top_d = []
    for el in fdist.most_common(15):
        top_d.append(el)
    return top_d, snt

##### Текст 1

In [22]:
text_1 = count_top_words(texts[0])

In [23]:
# текст 1 со всем мусором

texts[0]

'Эксперт утверждает, что Вашингтон может проиграть войну с Россией. В конце мая этого года специалисты Центра стратегических и бюджетных оценок (Center for Strategic and Budgetary Assessments, CSBS, ЦСБО), базирующегося в Вашингтоне и являющегося довольно крупной независимой организацией, работающей по заказам Пентагона, военно-промышленного комплекса (ВПК) США, а также проводящей исследования в интересах союзников и партнеров Америки, опубликовали доклад под названием "Планировка игрового поля: восстановление ракет США на ТВД после закрытия ДРСМД". Составители доклада, исходя из того, что этого соглашение благополучно скончалось, довольно тщательно изучили стратегические, оперативно-тактические, финансовые и другие вопросы, связанные с необходимостью развертывания ракет среднего и меньшего радиуса действия на сухопутных ТВД, и то, как это может повлиять на укрепление обороноспособности Америки. А 5 июля газета The New York Times разместила на своих страницах статью одного из них, неко

In [24]:
new_words = []

In [25]:
# слова для первого текста

words_1 = [
           'вашингтон', 'россия', 'война', 'цсбо', '5 июль', '30 год', 
           'пентагон', 'специалист', 'сша', 'военно-промышленный комплекс',
           'ракета', 'твд', 'доклад', 'радиус действия', 'сухопутный', 'рсмд',
           'обороноспособность', 'америка', 'статья',
           'сотрудник', 'дрсмд', 'соединенный штат', 'договор', 'базироваться',
           'запрещенный', 'средний дальность', 'вооружение', 'наземный базирование',
           'белый дом', 'нато', '100 единица', 'китай', 'отказываться', 
           'противодействие', 'боеголовка', 'атаковать', 'база', 'территория', 
           'боевой столкновение', 'поражение', 'вс', 'противник', 'боеспособность',
           'оружие', 'сдерживание', 'оснащение', 'переговоры', 'уолтон',
           'перепрофилирование', 'крылатый ракета', 'tomahawk', 'заключение договор',
           'партнер америка', 'ядерный', 'неядерный', 'оперативно-тактический'           
]

new_words.append(words_1)

In [26]:
lemm_text = []

In [27]:
# текст 1 без мусора

lemm_text.append(text_1[1])

In [28]:
# топ-15 слов

text_1[0]

[('ракета', 17),
 ('россия', 7),
 ('это', 7),
 ('мочь', 6),
 ('сша', 6),
 ('китай', 6),
 ('америка', 5),
 ('договор', 5),
 ('соединить', 5),
 ('штат', 5),
 ('дальность', 5),
 ('американский', 5),
 ('боеголовка', 5),
 ('год', 4),
 ('дрсмд', 4)]

In [29]:
# ключевые слова к тексту 1

keys_start[0]

{'100 единиц': ['NUSR', 1],
 '100 запрещенных': ['NUSR', 1],
 '1990-х': ['NUSR', 1],
 '30 лет': ['NUSR', 1],
 '5 июля': ['NUSR', 1],
 'c': ['NAME', 1],
 'csbs': ['UPPR', 1],
 'ii': ['ROMN', 1],
 'pershing': ['TITL', 1],
 't': ['ABVR', 1],
 'tnyt': ['NAME', 1],
 'tomahawk': ['TITL', 1],
 'азия': ['NOUN', 1],
 'аляска': ['NOUN', 1],
 'америка': ['NOUN', 5],
 'американский': ['ADJF', 5],
 'базирование': ['NOUN', 4],
 'белый': ['ADJF', 1],
 'боеголовка': ['NOUN', 5],
 'вашингтон': ['NOUN', 2],
 'впк': ['ABVR', 1],
 'вс': ['ABVR', 3],
 'гуам': ['NOUN', 1],
 'дальность': ['NOUN', 5],
 'договор': ['NOUN', 5],
 'дрсмд': ['UPPR', 3],
 'зд': ['ABVR', 1],
 'иванов': ['ADJF', 1],
 'китай': ['NOUN', 6],
 'комплекс': ['NOUN', 1],
 'корабль': ['NOUN', 2],
 'лодка': ['NOUN', 1],
 'марианский': ['ADJF', 1],
 'мо': ['ABVR', 1],
 'москва': ['NOUN', 1],
 'наземный': ['ADJF', 4],
 'нато': ['ABVR', 1],
 'нв': ['WIQM', 1],
 'нво': ['WIQM', 1],
 'пентагон': ['NOUN', 2],
 'планировка': ['NOUN', 1],
 'ракета': 

##### Текст 2

In [30]:
text_2 = count_top_words(texts[1])

In [31]:
texts[1]

' Военный эксперт Антон Лавров — о том, почему приобретение Украиной штурмовиков Super Tucano окажется ошибкой. Украина рассматривает вопрос закупки в Бразилии турбовинтовых учебно-боевых самолетов A-29 Super Tucano. Делегация ВВС этой страны во главе с командующим генерал-полковником Сергеем Дроздовым прибыла в Сан-Паулу и уже провела переговоры c руководством бразильской компании Embraer, производящей самолет. А главком ВВС даже обкатал турбовинтовую машину — лично поднялся на ней в небо. Но такие контракты не заключаются быстро, и одного визита командующего ВВС для принятия решения будет недостаточно. Те же США испытывают A-29 Super Tucano не первый год, но так и не могут решиться на их приобретение по программе Light Combat Aircraft. Конечно, желание Киева объяснимо. Замену советскому парку боевых самолетов Украина всерьез начала искать уже в 2015 году, потеряв изрядную часть ВВС в боях на востоке страны. Но бесплатно полк-другой современных многоцелевых истребителей им подарить ни

In [32]:
# слова для второго текста

words_2 = [
           'эксперт', 'украина', 'штурмовик', 'бразилия',
           'учебно-боевой самолет', 'ввс', 'a-29 super tucano',
           'турбовинтовый', 'контракт', 'приобретение', 'киев',
           'истребитель', 'цена', 'авиация', 'боевой возможность',
           'летный характеристика', 'снабжать', 'сша', 'афганистан',
           'вооружение', 'авиабомба', 'боевой вылет', 'боевой операция',
           'бомбовый нагрузка', 'пво', 'повстанец', 'ракетный комплекс',
           'угроза', 'зрк', 'противник', 'летчик', 'разведслужба',
           'техническая характеристика', 'оружие', 'оборонительный',
           'ракета', 'обзорно-прицельный станция', 'неэффективный',
           'эрзац-штурмовик', 'экономический выгода', 'беспилотник',
           'талиб', 'фугасный авиабомба', 'украинский', 'лазерный наведение',
           '30 год', 'md-530f', 'экипаж', 'самолет-платформа', 'приобрести'
]

new_words.append(words_2)

In [33]:
lemm_text.append(text_2[1])

In [34]:
# топ-15 слов

text_2[0]

[('самолёт', 14),
 ('год', 10),
 ('tucano', 9),
 ('украина', 8),
 ('super', 8),
 ('ввс', 7),
 ('боевой', 7),
 ('страна', 6),
 ('сша', 6),
 ('это', 6),
 ('лёгкий', 6),
 ('применение', 6),
 ('штурмовик', 5),
 ('мочь', 4),
 ('ещё', 4)]

In [35]:
# ключевые слова к тексту 2

# keys_start[1]

##### Текст 3

In [36]:
text_3 = count_top_words(texts[2])

In [37]:
texts[2]

' Военный эксперт спрогнозировал итоги войны США и России на море Черное море стало основным место противостояния России и США, пока - заочного, написал военный аналитик Лайл Голдштейн в статье для издания The National Interest. При этом он казал, что американские авианосцы имеют ограниченную полезность в гипотетической войне за будущее Украины, и призвал Пентагон осторожнее вести себя в Черном море. Черное море стало местом напряженного, но пока заочного противостояния России и США после задержания в Керченском проливе в ноябре прошлого года сотрудниками пограничной службы ФСБ РФ 12 украинских моряков на трех судах за нарушение российской госграницы, пишет военный аналитик Лайл Голдштейн в своей колонке для американского военно-политического издания The National Interest. При этом, по его словам, Вашингтон не помог решению вопроса, "подняв ставку" в противостоянии с Москвой военными учениями НАТО Sea Breeze, проводимыми совместно с Украиной и еще 19 странами в Одессе. Посредством этих

In [38]:
# слова для третьего текста

words_3 = [
           'военный', 'сша', 'россия', 'война', 'черный море',
           'место противостояние', 'американский', 'украина',
           'пентагон', 'противостояние', 'керченский пролив', 
           'фсб', 'задержание', 'моряк', 'госграница', 'нато',
           'демонстрация сила', 'киев', 'риск', 'переговоры',
           'мина', 'судно', 'авианосец', 'подводный лодка',
           'ракетный комплекс', 'ракетный катер', 'противокорабельный ракета',
           'вмс', 'конфликт', 'акватория', 'рф', 'sea breeze-2019',
           'учение', '2019 год', 'вмф', 'развертывание корабль', 
           'marinetraffic', 'шестой флот', 'вмс', 'российский',
           'оперативный реагирование', 'региональный стабильность',
           'the national interest', 'второй мировой война',
           'подорванный', 'фан', 'крым', 'азовский море', 
           'страна-участница', '32 корабль', '24 самолет', '3 тыс.',
           'военнослужащий', 'с воздух', 'противолодочный',
           'борт' 
]

new_words.append(words_3)

In [39]:
lemm_text.append(text_3[1])

In [40]:
# топ-15 слов

text_3[0]

[('море', 16),
 ('сша', 15),
 ('чёрный', 14),
 ('американский', 8),
 ('корабль', 8),
 ('военный', 6),
 ('россия', 6),
 ('война', 5),
 ('учение', 5),
 ('вмс', 5),
 ('флот', 5),
 ('это', 4),
 ('авианосец', 4),
 ('украина', 4),
 ('эсминец', 4)]

In [41]:
# ключевые слова к тексту 2

# keys_start[2]

##### Текст 4

In [42]:
text_4 = count_top_words(texts[3])

In [43]:
texts[3]

' В составе бригад управления Минобороны РФ формируют роты, оснащенные мобильными ретрансляторами и антеннами сотовой связи. Первые такие подразделения уже созданы на Дальнем Востоке и в Сибири. Они могут за считаные минуты создавать локальные сети сотовой связи, подключаясь к сигналу основных федеральных операторов в радиусе до 30–40 км от передвижной вышки. Такая техника нужна для формирования закрытых для противника зон обмена информацией, а также для налаживания устойчивой связи в районах стихийных бедствий. В составе бригад управления Минобороны РФ формируют роты, оснащенные мобильными ретрансляторами и антеннами сотовой связи. Первые такие подразделения уже созданы на Дальнем Востоке и в Сибири. Они могут за считаные минуты создавать локальные сети сотовой связи, подключаясь к сигналу основных федеральных операторов в радиусе до 30–40 км от передвижной вышки. Такая техника нужна для формирования закрытых для противника зон обмена информацией, а также для налаживания устойчивой св

In [44]:
# слова для четвертого текста

words_4 = [
           'миноборона', 'рота', 'мобильный ретранслятор',
           'дальный восток', 'сирия', 'антенна сотовая связь',
           'локальный сеть', 'передвижной вышка', 'зона обмен информация',
           'устойчивый связь', 'стихийный бедствие', 'десять автомобиль',
           'техника', 'хабаровск', 'новороссийск', 'аппаратура', 'камаз',
           'сигнал', 'электроенный устройство', 'герметизированный кузов',
           'оборудование', 'федеральный оператор', 'инфраструктура',
           'зона покрытие', 'учение', 'труднодоступный район',
           'ликвидация последствие', 'местный население',
           'вышка', 'рельеф', 'испытание', 'связист', 'интегрироваться',
           'передача данные', 'полевой условие', 'война', 'игил',
           'войско', 'спутниковый', 'радиорелйеный', 'gsm', 'асув',
           'ов', 'военный коммуникация', 'телефон', 'криптографиский', 'спецблок',
           'шифрование', 'рф', '30-40 км', 'интернет-мессенджер',
           'армия', 'категория секретность', 'канал связь', 'отсутствовать инфраструктура',
           'wi-fi', 'армейский командир', 'пострадать', 'дистанционный активация',
           'экстренный стирание', 'мирный время'
]

new_words.append(words_4)

In [45]:
lemm_text.append(text_4[1])

In [46]:
# топ-15 слов

text_4[0]

[('связь', 20),
 ('сотовый', 18),
 ('сеть', 8),
 ('мобильный', 7),
 ('оператор', 7),
 ('мочь', 6),
 ('вышка', 6),
 ('зона', 6),
 ('новый', 6),
 ('военный', 6),
 ('ретранслятор', 5),
 ('сигнал', 5),
 ('информация', 5),
 ('район', 5),
 ('управление', 4)]

In [47]:
# ключевые слова к тексту 4

# keys_start[3]

##### Текст 5

In [48]:
text_5 = count_top_words(texts[4])

In [49]:
texts[4]

' США и Россия расторгли ДРСМД, мир пошатнулся. Между тем у Германии, мечтающей об особом пути, есть разоренная армия и нет собственной стратегии. С 1990-го года Германия не использовала период "междуцарствия", чтобы в глобализированном мире при чрезмерной усталости США определить свои интересы и найти новые балансы — прежде всего с США, а затем и с Россией и с Китаем, пишет немецкая "Вельт". Природа и политика не терпят вакуума. Мировому порядку нужна мировая держава. Иначе, как это вышло сейчас, борьба за равновесие или гегемонию превратится в ядерную игру в кости. Это причина, по которой США, о чем Дональд Трамп постоянно пишет в своих противоречивых твитах, не могут освободиться от обязательств и интересов, военных инвестиций и обещаний гарантий. Ведущая роль США как ядерной супердержавы была и остается связанной с ДНЯО (Договор о нераспространении ядерного оружия), которому уже полвека. Если США задействуют свои "звезды и полосы" на Хоккайдо, в Ландштуле, на шотландских базах ядер

In [50]:
# слова для четвертого текста

words_5 = [
           'сша', 'россия', 'дрсмд', 'германия',
           'китай', 'политика', 'мировой порядок', 'мировой держава',
           'гегемония', 'военный инвестиция', 'ядерный',
           'супердержава', 'дняо', 'хоккайдо', 'ландштуль',
           'персидский залив', 'политик', 'берлин',
           'карибский кризис', 'война', 'мир', 'холодный война',
           'северный корея', 'иран', 'ядерный вопрос', '50-ый год',
           'распространение ядерный оружие', 'япония', 'отказ',
           'сдерживание', 'нато', 'боевой действие', 'гарантия',
           'американский мир', 'ядерный система', 'сокращение риск',
           'немец', 'стратегический мышление', 'европа', '1989',
           'америка', 'перерастяжение сила', 'элита', 'европеец',
           '1982-ой год', 'pax americana', 'парадигма государственный политика',
           'тайвань', 'вельт', 'борьба за равновесие', 'обещание гарантия',
           'угроза апокалипсис', 'твит', 'трамп', 'стратегический условие'
]

new_words.append(words_5)

In [51]:
lemm_text.append(text_5[1])

In [52]:
# топ-15 слов

text_5[0]

[('ядерный', 13),
 ('сша', 10),
 ('оружие', 9),
 ('мир', 8),
 ('свой', 8),
 ('мировой', 7),
 ('это', 7),
 ('который', 7),
 ('всё', 6),
 ('немецкий', 5),
 ('порядок', 5),
 ('держава', 5),
 ('россия', 4),
 ('год', 4),
 ('вопрос', 4)]

In [53]:
# ключевые слова к тексту 4

# keys_start[4]

In [54]:
df['lemm_texts'] = lemm_text
df['new_key_words'] = new_words

In [55]:
df['len_new_key_words'] = df['new_key_words'].apply(len)
df['len_keys'] = df['keys'].apply(len)

In [56]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г...",825,сша россия расторгнуть дрсмд мир пошатнуться г...,"[сша, россия, дрсмд, германия, китай, политика...",55,62


В результате:

* количество выделенных вручную слов меньше завявленного по той причине, что первоначальная разметка не учитывает словосочетания (маркированные выражения, в проекте, из которого брались результаты, учитывались именно ключевые СЛОВА, делая же это руками, я учитывала также маркированные выражаения наряду с ключевыми словами), в отличие от той, которая была сделана в домашней работе

* учитывая пункт выше, самым логичным вариантов будет взять пересечение ключевых слов и добавить к ним словосочетания, выделенные вручную, таким образом полуичтся составить эталон разметки: будет возможност опереться на уже выделенные слова и учитывать при этом те, что выделила я

* при этом, если смотреть на самые частотные слова заметно пересечение с тем, что выделили руками

In [57]:
# пересечения

interc = []

ind = 0
for el in only_keys:
    new_l = list(set(el) & set(new_words[ind]))
    interc.append(new_l)
    ind += 1

In [58]:
df['intersection_keys'] = interc
df['len_inters'] = df['intersection_keys'].apply(len)

In [59]:
# объединение списков

un = []

ind_2 = 0
for el in only_keys:
    new_l = list(set(el).union(set(new_words[ind_2])))
    un.append(new_l)
    ind_2 += 1

In [60]:
df['union_keys'] = un
df['len_union'] = df['union_keys'].apply(len)

Попробуем сделать "эталон" выбрав пересечение множеств и добавив к ним те элементы, где больше одного слова, из созданных нами ключевых слов

In [61]:
etal = []  # эталон

ind_3 = 0
for el in only_keys:
    new_l = list(set(new_words[ind_3]) - set(el))
    fin = []
    for el in new_l:  # берем разницу и словосочетания из ключевых слов
        if len(el.split(' ')) >= 2:
            fin.append(el)
    # объединение множества пересечений и разницы
    last = list(set(fin).union(set(interc[ind_3])))
    ind_3 += 1
    etal.append(last)

In [62]:
df['etalon_keys'] = etal
df['len_etalon'] = df['etalon_keys'].apply(len)

In [63]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys,intersection_keys,len_inters,union_keys,len_union,etalon_keys,len_etalon
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54,"[рсмд, договор, цсбо, америка, ракета, твд, to...",17,"[рсмд, крылатый ракета, твд, 30 год, сухопутны...",93,"[рсмд, крылатый ракета, цсбо, ракета, америка,...",30
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65,"[ракета, ввс, зрк, штурмовик, истребитель, md-...",16,"[бомбовый нагрузка, летный характеристика, 30 ...",99,"[ракета, бомбовый нагрузка, летный характерист...",30
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83,"[крым, военный, акватория, мина, marinetraffic...",19,"[крым, военный, национальный, шестой, противол...",119,"[крым, развертывание корабль, второй мировой в...",40
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52,"[техника, оборудование, вышка, сигнал, рф, кам...",13,"[военный, пострадать, спецблок, дистанционный ...",100,"[техника, аппаратура, армейский командир, дист...",40
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г...",825,сша россия расторгнуть дрсмд мир пошатнуться г...,"[сша, россия, дрсмд, германия, китай, политика...",55,62,"[америка, хоккайдо, мир, ядерный, дрсмд, иран,...",16,"[персидский залив, стратегический условие, зип...",101,"[персидский залив, стратегический условие, аме...",39


С одной стороны кажется странным, что для текстов длиной 600-800 в эталонных мы выделили только в районе 30-40 слов, но с другой стороны, при чтении текстов становится понятно, что в них очень много "воды" (в одной из статей зачем-то повторяют 3 раза один и тот же абзац (на сайте именно так), соотвествтенно, там не будет болього количества ключевых слов). Многие из слов статьи при этом являются слишком общими типа "сказал", "написал", "издание", "считает" и т.д. В данных новостных текстах этого очень-очень много, поэтому ключевых слов, которые были бы направлены на описание отражаемой ситуации дествительно немного.

Но вариант использовать в качестве эталона объединение тоже хорошо, так как таким образом получится учесть все: и словосочетания, собранные вручную, и то, что было выделено в проекте.

### 3: (2 балла) Применить к этому корпусу 3 метода извлечения ключевых слов на выбор (RAKE, TextRank, tf*idf, OKAPI BM25, ...)

##### RAKE

In [64]:
!pip install python-rake

In [65]:
import RAKE

In [66]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
# инициализируем стоп-слова еще раз
stop = stopwords.words('russian')

In [68]:
more_stops = ['это', "который"]

for el in more_stops:
    stop.append(el)

In [69]:
# Инициализируем анализатор списком стоп-слов
rake = RAKE.Rake(stop)

In [70]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [71]:
#  возьмем способ нормализации текста с семинара, чтобы было красиво
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [72]:
# Попробуем запустить на обычном тексты и на лемматизированном
# Сравним результаты

text_list_norm = rake.run(normalize_text(texts[0]), maxWords=3, minFrequency=2)

In [73]:
text_list_norm

[('обычный боеголовка', 4.2),
 ('россия', 1.4),
 ('китай', 1.0),
 ('море', 1.0),
 ('противник', 1.0),
 ('оружие', 1.0)]

In [74]:
text_list_ord = rake.run(texts[0], maxWords=3, minFrequency=2)

In [75]:
text_list_ord

[('обычными боеголовками', 4.2),
 ('россией', 1.0),
 ('китай', 1.0),
 ('китая', 1.0),
 ('море', 1.0),
 ('таких', 1.0)]

Очевидно, что результат более реальный получился на нормализованных текстах, поэтому будем использовать только его дальше. (возьмем минимальную частотность 1 чтобы выделялось больше 6 слов)

In [76]:
rake_l = []

for text in texts:
    lst = []
    text_list_norm = rake.run(normalize_text(text), maxWords=3, minFrequency=1)
    for el in text_list_norm:
        if el[0] not in lst:
            lst.append(el[0])
    rake_l.append(lst)

In [77]:
df['rake'] = rake_l
df['len_rake'] = df['rake'].apply(len)

##### TextRank

In [78]:
from gensim.summarization import keywords as kw

In [79]:
kw(normalize_text(texts[0]), pos_filter=[], scores=True)

[('ракета', 0.410104290146431),
 ('что', 0.27119520656416324),
 ('мочь', 0.15042229502963284),
 ('американскии', 0.13632390604492448),
 ('договор', 0.12233025273957998),
 ('такои', 0.12166788950653232),
 ('америка', 0.12126684792060521),
 ('это год специалист', 0.12044509396326812),
 ('сша', 0.11938587790733544),
 ('россия', 0.11583230880766378),
 ('для', 0.11285130408208607),
 ('более', 0.10259799337951758),
 ('оснастить', 0.09738700773431948),
 ('дрсмд', 0.0947445739960929),
 ('боеголовка', 0.08989948187349908),
 ('довольно', 0.08431439444049776),
 ('этот', 0.08283296146559645),
 ('китаи', 0.08127721280175769),
 ('вероятность', 0.08082142247015059),
 ('цсбыи', 0.0803275682257977),
 ('время', 0.07962279183453454),
 ('тимоти уолтон', 0.07606889405685355),
 ('доклад под', 0.07575008894492019),
 ('соединить штат соблюдать условие', 0.0729686223433895),
 ('деиствие', 0.07242564700240367),
 ('дальность', 0.07200489964614361),
 ('которыи', 0.0719697195757566),
 ('модель', 0.0703815781940490

Из-за того, что тут не учитываются стоп-слова получается проблема - некоторые ключевые слова являются одновременно и стоп-словами. Мы можем это убрать, что я собственно и хочу сделать для всех текстов.

In [80]:
text_rank_l = []

for text in texts:
    lst = []
    text_list_norm = kw(normalize_text(text), pos_filter=[], scores=True)
    for el in text_list_norm:
        if el[0] not in lst and el[0] not in ru_stops:
            lst.append(el[0])
    text_rank_l.append(lst)

In [81]:
df['text_rank'] = text_rank_l
df['len_text_rank'] = df['text_rank'].apply(len)

##### Tf-Idf

In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [83]:
import numpy as np

Так как у нас везде в среднем около 50 слов на тест, то возьмем для тф-айдф топ 45

In [84]:
tf_idf_l = []

for text in texts:
    vectorizer = TfidfVectorizer(stop_words=stop)
    X = vectorizer.fit_transform([normalize_text(text)])
    names = vectorizer.get_feature_names()
    data_1 = X.todense().tolist()  # результаты
    for_t = pd.DataFrame(data_1, columns=names)
    for i in for_t.iterrows():
        lst = i[1].sort_values(ascending=False)[:45].index.tolist()
    tf_idf_l.append(lst)

In [85]:
df['tf_idf'] = tf_idf_l
df['len_tf_idf'] = df['tf_idf'].apply(len)

In [86]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys,intersection_keys,len_inters,union_keys,len_union,etalon_keys,len_etalon,rake,len_rake,text_rank,len_text_rank,tf_idf,len_tf_idf
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54,"[рсмд, договор, цсбо, америка, ракета, твд, to...",17,"[рсмд, крылатый ракета, твд, 30 год, сухопутны...",93,"[рсмд, крылатый ракета, цсбо, ракета, америка,...",30,"[военно-промышленный комплекс, соглашение благ...",138,"[ракета, мочь, американскии, договор, такои, а...",44,"[ракета, россия, китай, сша, мочь, соединить, ...",45
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65,"[ракета, ввс, зрк, штурмовик, истребитель, md-...",16,"[бомбовый нагрузка, летный характеристика, 30 ...",99,"[ракета, бомбовый нагрузка, летный характерист...",30,"[совершенный обзорно-прицельный станция, закуп...",183,"[самолет, год, боевои, сша, легкии, ввс этот с...",56,"[самолёт, год, tucano, боевой, украина, super,...",45
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83,"[крым, военный, акватория, мина, marinetraffic...",19,"[крым, военный, национальный, шестой, противол...",119,"[крым, развертывание корабль, второй мировой в...",40,"[вице-адмирал лиза франчетти, нарушение россий...",120,"[море, воина сша, американскии авианосец, кора...",48,"[море, сша, чёрный, корабль, американский, вое...",45
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52,"[техника, оборудование, вышка, сигнал, рф, кам...",13,"[военный, пострадать, спецблок, дистанционный ...",100,"[техника, аппаратура, армейский командир, дист...",40,"[помощь обычный интернет-мессенджер, антенный ...",147,"[сотовыи связь, новыи, зона, раион, военныи ил...",48,"[связь, сотовый, сеть, оператор, мобильный, мо...",45
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г...",825,сша россия расторгнуть дрсмд мир пошатнуться г...,"[сша, россия, дрсмд, германия, китай, политика...",55,62,"[америка, хоккайдо, мир, ядерный, дрсмд, иран,...",16,"[персидский залив, стратегический условие, зип...",101,"[персидский залив, стратегический условие, аме...",39,"[музей трагический заблуждение, настать час по...",217,"[ядерныи, мир, как это, сша, немецкии, свои, к...",55,"[ядерный, сша, оружие, свой, мир, мировой, всё...",45


### 4: (2 балла) Составить морфологические/синтаксические шаблоны для ключевых слов и фраз, выделить соответствующие им подстроки из корпуса (например, именные группы Adj+Noun). Применить эти фильтры к спискам ключевых слов.

Сначала я хотела прогнать текст через udpipe и разметить весь текст, но потом подумала, что есть алгоритм проще:

пройтись по списку ключевых слов теггером (мне кажется, что в таком случае не очень оптимален будет парсер), оставить только то, что подходит под эталонные шаблоны



---

С униграммами может быть проблема, потому что они захватывают очень многие слова в тексте в приницпе и могут не быть ключевыми в приницпе (


In [87]:
# посмотрим, какой максимальный размер н-грамм есть в эталонах

max = 0

for el in etal:
    for w in el:
        n_g = len(w.split(' '))
        if n_g > max:
            max = n_g

In [88]:
max  # максимальный н-грамм - 3

3

In [89]:
from nltk.util import ngrams

In [102]:
# оставим текст с пунктуацией, это будет важно
# с точки зрения синтаксииса (ничего не делим для получения н-грамм)

n_g = max
# пример на первых 100 символах
tokens_1 = list(tokenize(str(normalize_text(texts[0][:100]))))
tokens = [_.text for _ in tokens_1]
output = list(ngrams(tokens, n_g))

In [103]:
output

[('эксперт', 'утверждать', ','),
 ('утверждать', ',', 'что'),
 (',', 'что', 'вашингтон'),
 ('что', 'вашингтон', 'мочь'),
 ('вашингтон', 'мочь', 'проиграть'),
 ('мочь', 'проиграть', 'война'),
 ('проиграть', 'война', 'с'),
 ('война', 'с', 'россия'),
 ('с', 'россия', '.'),
 ('россия', '.', 'в'),
 ('.', 'в', 'конец'),
 ('в', 'конец', 'май'),
 ('конец', 'май', 'это'),
 ('май', 'это', 'год'),
 ('это', 'год', 'специалист')]

In [106]:
# теггер с universal tags
!pip install --quiet rnnmorph

     |████████████████████████████████| 10.5MB 2.6MB/s 


In [107]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

In [114]:
# пример применения н-грамм

for el in output:
    pr = predictor.predict(el)
    n_grm = []
    for w in pr:
        n_grm.append(w.pos)
    print(el, n_grm)

('эксперт', 'утверждать', ',') ['NOUN', 'VERB', 'PUNCT']
('утверждать', ',', 'что') ['VERB', 'PUNCT', 'PRON']
(',', 'что', 'вашингтон') ['PUNCT', 'PRON', 'NOUN']
('что', 'вашингтон', 'мочь') ['CONJ', 'NOUN', 'VERB']
('вашингтон', 'мочь', 'проиграть') ['NOUN', 'VERB', 'VERB']
('мочь', 'проиграть', 'война') ['VERB', 'VERB', 'NOUN']
('проиграть', 'война', 'с') ['VERB', 'NOUN', 'ADP']
('война', 'с', 'россия') ['NOUN', 'ADP', 'NOUN']
('с', 'россия', '.') ['ADP', 'NOUN', 'PUNCT']
('россия', '.', 'в') ['NOUN', 'PUNCT', 'ADP']
('.', 'в', 'конец') ['PUNCT', 'ADP', 'NOUN']
('в', 'конец', 'май') ['ADP', 'NOUN', 'NOUN']
('конец', 'май', 'это') ['NOUN', 'NOUN', 'PART']
('май', 'это', 'год') ['NOUN', 'PART', 'NOUN']
('это', 'год', 'специалист') ['PART', 'NOUN', 'NOUN']


Посмотрим на слова, которые мы считаем эталонными и выделим для них униграммы, би-граммы и три-граммы, потом надем внутри текстов все штуки, которые соответствуют им

In [157]:
# функция для предсказания строчки по н-граммам

def predict_pos(sent, n):
    pr = predictor.predict(sent)
    n_grm = ''
    for w in pr:
        if w.pos == 'PUNCT':  
            n_grm += 'ENG' + '_'
        else:
            n_grm += w.pos + '_'
    return n_grm

In [158]:
# получаем н-граммы

all_ngrams = []
for text in etal:
    dct = {}
    uni = []
    big = []
    tre = []
    for word in text:
        if len(word.split(' ')) == 1:
            if predict_pos([word], 1) not in uni:
                uni.append(predict_pos([word], 1))
        elif len(word.split(' ')) == 2:
            if predict_pos(word.split(' '), 2) not in big:
                big.append(predict_pos(word.split(' '), 2))
        elif len(word.split(' ')) == 3:
            if predict_pos(word.split(' '), 3) not in tre:
                tre.append(predict_pos(word.split(' '), 3))
    dct['униграммы'] = uni
    dct['биграммы'] = big
    dct['триграммы'] = tre
    all_ngrams.append(dct)

In [159]:
df['etalon_n_grams'] = all_ngrams

In [160]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys,intersection_keys,len_inters,union_keys,len_union,etalon_keys,len_etalon,rake,len_rake,text_rank,len_text_rank,tf_idf,len_tf_idf,etalon_n_grams
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54,"[рсмд, договор, цсбо, америка, ракета, твд, to...",17,"[рсмд, крылатый ракета, твд, 30 год, сухопутны...",93,"[рсмд, крылатый ракета, цсбо, ракета, америка,...",30,"[военно-промышленный комплекс, соглашение благ...",138,"[ракета, мочь, американскии, договор, такои, а...",44,"[ракета, россия, китай, сша, мочь, соединить, ...",45,"{'униграммы': ['NOUN_', 'ADV_', 'ENG_', 'ADP_'..."
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65,"[ракета, ввс, зрк, штурмовик, истребитель, md-...",16,"[бомбовый нагрузка, летный характеристика, 30 ...",99,"[ракета, бомбовый нагрузка, летный характерист...",30,"[совершенный обзорно-прицельный станция, закуп...",183,"[самолет, год, боевои, сша, легкии, ввс этот с...",56,"[самолёт, год, tucano, боевой, украина, super,...",45,"{'униграммы': ['NOUN_', 'NUM_', 'ADP_', 'ADJ_'..."
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83,"[крым, военный, акватория, мина, marinetraffic...",19,"[крым, военный, национальный, шестой, противол...",119,"[крым, развертывание корабль, второй мировой в...",40,"[вице-адмирал лиза франчетти, нарушение россий...",120,"[море, воина сша, американскии авианосец, кора...",48,"[море, сша, чёрный, корабль, американский, вое...",45,"{'униграммы': ['NOUN_', 'ADJ_', 'ENG_'], 'бигр..."
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52,"[техника, оборудование, вышка, сигнал, рф, кам...",13,"[военный, пострадать, спецблок, дистанционный ...",100,"[техника, аппаратура, армейский командир, дист...",40,"[помощь обычный интернет-мессенджер, антенный ...",147,"[сотовыи связь, новыи, зона, раион, военныи ил...",48,"[связь, сотовый, сеть, оператор, мобильный, мо...",45,"{'униграммы': ['NOUN_', 'ADP_', 'ADJ_'], 'бигр..."
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г...",825,сша россия расторгнуть дрсмд мир пошатнуться г...,"[сша, россия, дрсмд, германия, китай, политика...",55,62,"[америка, хоккайдо, мир, ядерный, дрсмд, иран,...",16,"[персидский залив, стратегический условие, зип...",101,"[персидский залив, стратегический условие, аме...",39,"[музей трагический заблуждение, настать час по...",217,"[ядерныи, мир, как это, сша, немецкии, свои, к...",55,"[ядерный, сша, оружие, свой, мир, мировой, всё...",45,"{'униграммы': ['NOUN_', 'ADJ_'], 'биграммы': [..."


Изначально задание складывается из того, чтобы подумать о том, какие текстовые признаки/шаблоны могут быть полезны и быть сконструированы для текстов. А после внутри текста найти все н-граммы, котрые этим шаблонам соответствуют. Поэтому сейчас отберем все, что может влиять и поищем это в текстах

In [161]:
all_ngrams

[{'биграммы': ['ADJ_NOUN_', 'NUM_NOUN_', 'NOUN_NOUN_'],
  'триграммы': [],
  'униграммы': ['NOUN_', 'ADV_', 'ENG_', 'ADP_']},
 {'биграммы': ['ADJ_NOUN_', 'NUM_NOUN_'],
  'триграммы': ['NUM_VERB_ENG_'],
  'униграммы': ['NOUN_', 'NUM_', 'ADP_', 'ADJ_']},
 {'биграммы': ['NOUN_NOUN_',
   'ADJ_NOUN_',
   'ADV_NUM_',
   'NUM_NOUN_',
   'NUM_ENG_',
   'ADP_NOUN_'],
  'триграммы': ['ADJ_ADJ_NOUN_', 'INTJ_ENG_NUM_'],
  'униграммы': ['NOUN_', 'ADJ_', 'ENG_']},
 {'биграммы': ['ADJ_NOUN_', 'NOUN_NOUN_', 'NUM_NOUN_', 'VERB_NOUN_'],
  'триграммы': ['NOUN_ADJ_NOUN_', 'NOUN_NOUN_NOUN_'],
  'униграммы': ['NOUN_', 'ADP_', 'ADJ_']},
 {'биграммы': ['ADJ_NOUN_', 'NOUN_NOUN_', 'CONJ_ADV_'],
  'триграммы': ['NOUN_ADJ_NOUN_', 'NOUN_ADP_NOUN_'],
  'униграммы': ['NOUN_', 'ADJ_']}]

Все тексты написаны на одну и ту же тематику, поэтому мы можем проанализировать те н-граммы, которые нам встретились, отобрать лучшие и потом вытащить из каждого текста те, которые мы считаем важными для данной задачи.



---




*   **Триграммы**: "парадигма государственный политика" ('NOUN_ADJ_NOUN_'), "борьба за равновесие" ('NOUN_ADP_NOUN_'), "зона обмен информация" (NOUN_NOUN_NOUN_) кажутся очень важными. Это такие именные группы, в которых есть несколько существительных, которые могут кодировать важную информацию в текстах, например отношения между существительными. ADJ_ADJ_NOUN_ - тоже может быть важным, потому что такие группы могут выделять сложную конкотенированную информацию о событии. Вместе с этим триграграммы 'NUM_VERB_ENG_'/INTJ_ENG_NUM_ будут плохим варинатом, так как у них неправильная разметка  (a-29 super tucano, the national interest) (не столько важна разметка, сколько сами конструкции: первая кажется странной, если брать числительной, глагол и какое-то английское слово --> бессмыслица, вторая - междометие, английское слово и числительное тоже бесмысленны)


*   **Биграммы**: наличие союза или любой другой вспомогательной части речи в биграмме в данной задаче кажется странным. Нам интересны разные конструкции числительных+существительнх, существительных,существительных+наречий, существительных+прилагательных. Вот в этой задаче может быть очень интересно посмотреть на английские слова (чаще всего они встречаются в данных текстах вместе с числительными, поэтому оставим так). Вариант брать биграммы с существительным и глаголом кажется мне не очень хорошим потому, что с большей вероятностью мы вытащим просто подлежащее и сказуемое, чего мы точно не хотим, так как нам интересна индивидуальность и явная особенность ключевых слов/конструкций (включающая, например, дополнительную информацию и т.д.). Но тут есть вероятность наткнуться на кучу мусора в принципе, потому что могут доставаться только простые словосочетания, слабо связанные с ключевыми словами


* **Униграммы**: в приницпе тут важнее всего английские слова, существительное, глагол (прилагательные, но этого как-то меньше). Остальное мне кажется рассматривать немного быссмысленно

In [181]:
#  возвращаем подстроки из корпуса, которые соответствуют шаблонам

def string_pred(text, n, grams):
    tokens_1 = list(tokenize(str(normalize_text(text))))
    tokens = [_.text for _ in tokens_1]
    output = list(ngrams(tokens, n))
    res = []
    for el in output:
        pr = predictor.predict(el)
        col = ''
        for w in pr:
            if w.pos == 'PUNCT' and len(w.normal_form) >= 2:  
                col += 'ENG' + '_'
            else:
                col += w.pos + '_'
        new = ''
        if col in grams:
            new += ' '.join([f for f in list(el)])
            res.append(new)
    return res

In [174]:
# ищем триграммы в текстах
grams_3 = ['NOUN_ADJ_NOUN_', 'NOUN_ADP_NOUN_', 'NOUN_NOUN_NOUN_', 'ADJ_ADJ_NOUN_']

n_grams_texts = []
for text in texts:
    print(text[:50])
    n3 = string_pred(text, 3, grams_3)
    print(n3[:20])
    n_grams_texts.append(n3)

Эксперт утверждает, что Вашингтон может проиграть 
['война с россия', 'год специалист центр', 'крупный независимый организация', 'исследование в интерес', 'доклад под название', 'планировка игровой поле', 'восстановление ракета сша', 'сша на твд', 'необходимость развёртывание ракета', 'укрепление обороноспособность америка', 'год действие договор', 'договор о рсмд', 'ракета средний дальность', 'министерство оборона сша', 'вооружение неядерный ракета', 'ракета средний дальность', 'дальность наземный базирование', 'дипломат в течение', 'призыв белый дом', 'часть двусторонний договор']
 Военный эксперт Антон Лавров — о том, почему прио
['эксперт антон лавров', 'приобретение украина штурмовик', 'закупка в бразилия', 'бразилия турбовинтовой учебно-боевой', 'турбовинтовой учебно-боевой самолёт', 'страна в глава', 'руководство бразильский компания', 'визит командующий ввс', 'ввс для принятие', 'приобретение по программа', 'замена советский парка', 'парка боевой самолёт', 'ввс в бой', 'бой на 

In [177]:
grams_2 = ['ADJ_NOUN_', 'NOUN_NOUN_', 'NUM_NOUN_',
           'ADV_NUM_', 'NUM_ENG_', 'ADP_NOUN_']

n_grams_texts_2 = []
for text in texts:
    print(text[:50])
    n3 = string_pred(text, 2, grams_2)
    print(n3[:20])
    n_grams_texts_2.append(n3)

Эксперт утверждает, что Вашингтон может проиграть 
['с россия', 'в конец', 'конец май', 'год специалист', 'специалист центр', 'бюджетный оценка', 'в вашингтон', 'независимый организация', 'по заказ', 'заказ пентагон', 'военно-промышленный комплекс', 'в интерес', 'интерес союзник', 'партнёр америка', 'под название', 'игровой поле', 'восстановление ракета', 'ракета сша', 'на твд', 'после закрытие']
 Военный эксперт Антон Лавров — о том, почему прио
['военный эксперт', 'эксперт антон', 'антон лавров', 'приобретение украина', 'украина штурмовик', 'вопрос закупка', 'в бразилия', 'учебно-боевой самолёт', 'делегация ввс', 'в глава', 'командующий генерал-полковник', 'генерал-полковник сергей', 'в сан-паулу', 'бразильский компания', 'главком ввс', 'турбовинтовой машина', 'в небо', 'один визит', 'командующий ввс', 'для принятие']
 Военный эксперт спрогнозировал итоги войны США и 
['военный эксперт', 'итог война', 'война сша', 'на море', 'чёрный море', 'основный место', 'место противостояние', 'п

In [182]:
grams_1 = ['NOUN_', 'ADJ_', 'ENG_', 'VERB_']

n_grams_texts_1 = []
for text in texts:
    print(text[:50])
    n3 = string_pred(text, 1, grams_1)
    print(n3[:20])
    n_grams_texts_1.append(n3)

Эксперт утверждает, что Вашингтон может проиграть 
['эксперт', 'утверждать', 'вашингтон', 'мочь', 'проиграть', 'война', 'россия', 'конец', 'май', 'год', 'специалист', 'центр', 'стратегический', 'бюджетный', 'оценка', 'for', 'strategic', 'csbs', 'цсбый', 'базироваться']
 Военный эксперт Антон Лавров — о том, почему прио
['военный', 'эксперт', 'антон', 'лавров', 'приобретение', 'украина', 'штурмовик', 'tucano', 'оказаться', 'ошибка', 'украина', 'рассматривать', 'вопрос', 'закупка', 'бразилия', 'турбовинтовой', 'учебно-боевой', 'самолёт', 'tucano', 'делегация']
 Военный эксперт спрогнозировал итоги войны США и 
['военный', 'эксперт', 'спрогнозировать', 'итог', 'война', 'сша', 'россия', 'море', 'чёрный', 'море', 'стать', 'основный', 'место', 'противостояние', 'россия', 'сша', 'заочный', 'написать', 'военный', 'аналитик']
 В составе бригад управления Минобороны РФ формиру
['состав', 'бригада', 'управление', 'минобороны', 'рф', 'формировать', 'рота', 'оснастить', 'мобильный', 'ретранслятор',

In [183]:
df['триграммы'] = n_grams_texts
df['биграммы'] = n_grams_texts_2
df['униграммы'] = n_grams_texts_1

In [184]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys,intersection_keys,len_inters,union_keys,len_union,etalon_keys,len_etalon,rake,len_rake,text_rank,len_text_rank,tf_idf,len_tf_idf,etalon_n_grams,триграммы,биграммы,униграммы
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54,"[рсмд, договор, цсбо, америка, ракета, твд, to...",17,"[рсмд, крылатый ракета, твд, 30 год, сухопутны...",93,"[рсмд, крылатый ракета, цсбо, ракета, америка,...",30,"[военно-промышленный комплекс, соглашение благ...",138,"[ракета, мочь, американскии, договор, такои, а...",44,"[ракета, россия, китай, сша, мочь, соединить, ...",45,"{'униграммы': ['NOUN_', 'ADV_', 'ENG_', 'ADP_'...","[война с россия, год специалист центр, крупный...","[с россия, в конец, конец май, год специалист,...","[эксперт, утверждать, вашингтон, мочь, проигра..."
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65,"[ракета, ввс, зрк, штурмовик, истребитель, md-...",16,"[бомбовый нагрузка, летный характеристика, 30 ...",99,"[ракета, бомбовый нагрузка, летный характерист...",30,"[совершенный обзорно-прицельный станция, закуп...",183,"[самолет, год, боевои, сша, легкии, ввс этот с...",56,"[самолёт, год, tucano, боевой, украина, super,...",45,"{'униграммы': ['NOUN_', 'NUM_', 'ADP_', 'ADJ_'...","[эксперт антон лавров, приобретение украина шт...","[военный эксперт, эксперт антон, антон лавров,...","[военный, эксперт, антон, лавров, приобретение..."
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83,"[крым, военный, акватория, мина, marinetraffic...",19,"[крым, военный, национальный, шестой, противол...",119,"[крым, развертывание корабль, второй мировой в...",40,"[вице-адмирал лиза франчетти, нарушение россий...",120,"[море, воина сша, американскии авианосец, кора...",48,"[море, сша, чёрный, корабль, американский, вое...",45,"{'униграммы': ['NOUN_', 'ADJ_', 'ENG_'], 'бигр...","[итог война сша, россия на море, море чёрный м...","[военный эксперт, итог война, война сша, на мо...","[военный, эксперт, спрогнозировать, итог, войн..."
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52,"[техника, оборудование, вышка, сигнал, рф, кам...",13,"[военный, пострадать, спецблок, дистанционный ...",100,"[техника, аппаратура, армейский командир, дист...",40,"[помощь обычный интернет-мессенджер, антенный ...",147,"[сотовыи связь, новыи, зона, раион, военныи ил...",48,"[связь, сотовый, сеть, оператор, мобильный, мо...",45,"{'униграммы': ['NOUN_', 'ADP_', 'ADJ_'], 'бигр...","[состав бригада управление, бригада управление...","[в состав, состав бригада, бригада управление,...","[состав, бригада, управление, минобороны, рф, ..."
4,"США и Россия расторгли ДРСМД, мир пошатнулся....","{'сша': ['NOUN', 10], 'россия': ['NOUN', 4], '...","[сша, россия, дрсмд, мир, германия, 1990-го, г...",825,сша россия расторгнуть дрсмд мир пошатнуться г...,"[сша, россия, дрсмд, германия, китай, политика...",55,62,"[америка, хоккайдо, мир, ядерный, дрсмд, иран,...",16,"[персидский залив, стратегический условие, зип...",101,"[персидский залив, стратегический условие, аме...",39,"[музей трагический заблуждение, настать час по...",217,"[ядерныи, мир, как это,

Сделаем теперь так, чтобы отобранные нами слова на разных методах встречлись в наших н-граммах. То есть, это нужно для задачи оценки точности с учетом морфо-синтаксических параметров (если внутри метода воозращаются какие-то слова, то они должны обязательно встретиться в нграммах)

In [198]:
# фильтруем ключевые слова

update_rake = []

i1 = 0
for tx in rake_l:
    new = []
    for el in tx:
        if el in n_grams_texts[i1] or el in n_grams_texts_2[i1] or el in n_grams_texts_1[i1]:
            new.append(el)
    update_rake.append(new) 
    i1 += 1

In [200]:
update_rank = []

i2 = 0
for tx in text_rank_l:
    new = []
    for el in tx:
        if el in n_grams_texts[i2] or el in n_grams_texts_2[i2] or el in n_grams_texts_1[i2]:
            new.append(el)
    update_rank.append(new) 
    i2 += 1

In [201]:
update_tf = []

i3 = 0
for tx in tf_idf_l:
    new = []
    for el in tx:
        if el in n_grams_texts[i3] or el in n_grams_texts_2[i3] or el in n_grams_texts_1[i3]:
            new.append(el)
    update_tf.append(new) 
    i3 += 1

In [204]:
df['rake_update'] = update_rake
df['rake_update_len'] = df['rake_update'].apply(len)
df['text_rank_update'] = update_rank
df['text_rank_update_len'] = df['text_rank_update'].apply(len)
df['tf_idf_update'] = update_tf
df['tf_idf_update_len'] = df['tf_idf_update'].apply(len)

In [205]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys,intersection_keys,len_inters,union_keys,len_union,etalon_keys,len_etalon,rake,len_rake,text_rank,len_text_rank,tf_idf,len_tf_idf,etalon_n_grams,триграммы,биграммы,униграммы,rake_update,text_rank_update,tf_idf_update,rake_update_len,text_rank_update_len,tf_idf_update_len
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54,"[рсмд, договор, цсбо, америка, ракета, твд, to...",17,"[рсмд, крылатый ракета, твд, 30 год, сухопутны...",93,"[рсмд, крылатый ракета, цсбо, ракета, америка,...",30,"[военно-промышленный комплекс, соглашение благ...",138,"[ракета, мочь, американскии, договор, такои, а...",44,"[ракета, россия, китай, сша, мочь, соединить, ...",45,"{'униграммы': ['NOUN_', 'ADV_', 'ENG_', 'ADP_'...","[война с россия, год специалист центр, крупный...","[с россия, в конец, конец май, год специалист,...","[эксперт, утверждать, вашингтон, мочь, проигра...","[военно-промышленный комплекс, призыв белый до...","[ракета, мочь, договор, америка, сша, россия, ...","[ракета, россия, китай, сша, мочь, соединить, ...",94,25,42
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65,"[ракета, ввс, зрк, штурмовик, истребитель, md-...",16,"[бомбовый нагрузка, летный характеристика, 30 ...",99,"[ракета, бомбовый нагрузка, летный характерист...",30,"[совершенный обзорно-прицельный станция, закуп...",183,"[самолет, год, боевои, сша, легкии, ввс этот с...",56,"[самолёт, год, tucano, боевой, украина, super,...",45,"{'униграммы': ['NOUN_', 'NUM_', 'ADP_', 'ADJ_'...","[эксперт антон лавров, приобретение украина шт...","[военный эксперт, эксперт антон, антон лавров,...","[военный, эксперт, антон, лавров, приобретение...","[совершенный обзорно-прицельный станция, ниша ...","[год, сша, применение, ракета, противник, маши...","[самолёт, год, tucano, боевой, украина, штурмо...",119,28,38
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83,"[крым, военный, акватория, мина, marinetraffic...",19,"[крым, военный, национальный, шестой, противол...",119,"[крым, развертывание корабль, второй мировой в...",40,"[вице-адмирал лиза франчетти, нарушение россий...",120,"[море, воина сша, американскии авианосец, кора...",48,"[море, сша, чёрный, корабль, американский, вое...",45,"{'униграммы': ['NOUN_', 'ADJ_', 'ENG_'], 'бигр...","[итог война сша, россия на море, море чёрный м...","[военный эксперт, итог война, война сша, на мо...","[военный, эксперт, спрогнозировать, итог, войн...","[вице-адмирал лиза франчетти, нарушение россий...","[море, корабль вмс, флот, эсминец, проводить, ...","[море, сша, чёрный, корабль, американский, вое...",82,26,39
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52,"[техника, оборудование, вышка, сигнал, рф, кам...",13,"[военный, пострадать, спецблок, дистанционный ...",100,"[техника, аппаратура, армейский командир, дист...",40,"[помощь обычный интернет-мессенджер, антенный ...",147,"[сотовыи связь, новыи, зона, раион, военныи ил...",48,"[связь, сотовый, сеть, оператор, мобильный, мо...",45,"{'униграммы': ['NOUN_', 'ADP_', 'ADJ_'], 'бигр...","[состав бригада управление, бригада управление...","[в состав, состав бригад

Шаблоны отсекают достаточно большую часть слов

### 5: (2 балла) Оценить точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них.

True Positive - количество верно распознанных ключевых слов (елемент списка есть и в другом списке)


False Negative- количество нераспознанных ключевых слов (элемент только эталонного списка, в предсказанном спике элемента нет)



False Positive - количество неверно распознанных ключевых слов (элемент списка есть только в предсказанном списке, но нет в эталонном)



In [290]:
# точность

def precision_f(tp, fp):
    func = tp / (tp + fp)
    return func

In [291]:
# полнота

def recall_f(tp, fn):
    func = tp / (tp + fn)
    return func

In [292]:
# f мера

def f_m(tp, fn, fp):
    func = (
        2 *  precision_f(tp, fp) * recall_f(tp, fn)) / (
        precision_f(tp, fp) + recall_f(tp, fn))
    return func

In [293]:
# функция для подсчета метрик качества


def metrics(df, etals, method):

    # получаем эталон и тот столбец, который мы сравниваем
    et = df[etals].tolist()
    real = df[method].tolist()
    
    # было бы удобно объеднить эти данные (zip)
    all_data = zip(et, real)

    # соберем информацию о количестве в списки
    # результатом для всех текстов будет среднее
    tp = []
    fp = []
    fn = []

    # в объединенных данных пройдемся и получим занчения для tp, fn, fp
    for etalon, pred in all_data:
        tp_num = 0
        fp_num = 0
        fn_num = 0
        for word in pred:
            if word in etalon:
                tp_num += 1
            if word not in etalon:
                fp_num += 1
        for wrd in etalon:
            if wrd not in pred:
                fn_num += 1
        tp.append(tp_num)
        fp.append(fp_num)
        fn.append(fn_num)
        
    return tp, fp, fn

In [294]:
def get_scores(df, etals, method):

    tp, fp, fn = metrics(df, etals, method)
    
    # для каждого текста
    prec = []
    rec = []
    f1 = []
    
    # собираем для каждого текста, а потом делаем mean
    for i in range(5):
        pr = precision_f(tp[i], fp[i])
        prec.append(pr)
        rc =  recall_f(tp[i], fn[i])
        rec.append(rc)
        f_s = f_m(tp[i], fn[i], fp[i])
        f1.append(f_s)
    
    # мы получили результаты предсказаний для каждого текста
    # считаем теперь для всех текстов --> среднее по оценкам
    

    print('Precision для корпуса: ', np.mean(np.array(prec)))
    print('Recall для корпуса: ', np.mean(np.array(rec)))
    print('F1-score для корпуса: ', np.mean(np.array(f1)))

In [295]:
# пример выдачи функции

get_scores(df, 'etalon_keys', 'tf_idf_update')

Precision для корпуса:  0.21378686308082973
Recall для корпуса:  0.24435897435897433
F1-score для корпуса:  0.22654354855175224


##### С учетом морфосинтаксического шаблона

Сначала попробуем на эталонных ключах (потом на объединении/пересечении, чтобы было)

###### RAKE

In [272]:
get_scores(df, 'etalon_keys', 'rake_update')

Precision для корпуса:  0.09684047682480686
Recall для корпуса:  0.2894871794871795
F1-score для корпуса:  0.14380701786822664


На пересечении

In [275]:
# rake

get_scores(df, 'intersection_keys', 'rake_update')

Precision для корпуса:  0.05275936479815224
Recall для корпуса:  0.34652298166230056
F1-score для корпуса:  0.09079467068137241


На объединении

In [278]:
# rake

get_scores(df, 'union_keys', 'rake_update')

Precision для корпуса:  0.23003690419027617
Recall для корпуса:  0.2343498226211887
F1-score для корпуса:  0.2286509011631242


Здесь лучший результат по F1 получился на объединенных ключах (там же самый высокий precision), самый высокий recall на пересечении

###### TextRank

In [273]:
get_scores(df, 'etalon_keys', 'text_rank_update')

Precision для корпуса:  0.2172967032967033
Recall для корпуса:  0.16897435897435897
F1-score для корпуса:  0.18930492292364473


На пересечении

In [276]:
# text rank
get_scores(df, 'intersection_keys', 'text_rank_update')

Precision для корпуса:  0.2172967032967033
Recall для корпуса:  0.35631400333412716
F1-score для корпуса:  0.268548199767712


На объединении

In [279]:
# text rank
get_scores(df, 'union_keys', 'text_rank_update')

Precision для корпуса:  0.4594945054945055
Recall для корпуса:  0.12278831439663349
F1-score для корпуса:  0.19355780622905533


Самый высокий F1-score на пересечении (там же самый высокий recall), самый высокий precision - на объединении

###### TF-IDF

In [274]:
get_scores(df, 'etalon_keys', 'tf_idf_update')

Precision для корпуса:  0.21378686308082973
Recall для корпуса:  0.24435897435897433
F1-score для корпуса:  0.22654354855175224


на пересечении

In [277]:
# tf-idf
get_scores(df, 'intersection_keys', 'tf_idf_update')

Precision для корпуса:  0.21378686308082973
Recall для корпуса:  0.5087133841390807
F1-score для корпуса:  0.3002154432146823


на объединении

In [280]:
# tf-idf
get_scores(df, 'union_keys', 'tf_idf_update')

Precision для корпуса:  0.5274681811267177
Recall для корпуса:  0.20446300872971546
F1-score для корпуса:  0.2940188018900868


Самый высокий F1-score на пересечении (+ самый высокий recall), самый высокий precision на объединении (по аналогии с текстранк)

##### Без морфосинтаксического шаблона

Тоже самое сделаем и здесь

###### RAKE

In [281]:
get_scores(df, 'etalon_keys', 'rake')

Precision для корпуса:  0.06836181473256077
Recall для корпуса:  0.3062820512820513
F1-score для корпуса:  0.11078817266343158


На пересечении

In [282]:
get_scores(df, 'intersection_keys', 'rake')

Precision для корпуса:  0.038132639531797366
Recall для корпуса:  0.3736723029292689
F1-score для корпуса:  0.06872113297571716


На объединении

In [283]:
get_scores(df, 'union_keys', 'rake')

Precision для корпуса:  0.1599139489332164
Recall для корпуса:  0.24448095831540323
F1-score для корпуса:  0.19000672750118786


Лучший F1-score на объединении + лучший precision

Лучший recall на пересечении

###### TextRank

In [284]:
get_scores(df, 'etalon_keys', 'text_rank')

Precision для корпуса:  0.11764069264069263
Recall для корпуса:  0.16897435897435897
F1-score для корпуса:  0.13788136118665562


На пересечении

In [285]:
get_scores(df, 'intersection_keys', 'text_rank')

Precision для корпуса:  0.11764069264069263
Recall для корпуса:  0.35631400333412716
F1-score для корпуса:  0.17601176753039238


На объединении

In [286]:
get_scores(df, 'union_keys', 'text_rank')

Precision для корпуса:  0.25801948051948054
Recall для корпуса:  0.12648918868574305
F1-score для корпуса:  0.16930026831392925


Лучший F1-score на пересечении + лучший recall

Лучший precision на объединении

###### TF-IDF

In [287]:
get_scores(df, 'etalon_keys', 'tf_idf')

Precision для корпуса:  0.19111111111111112
Recall для корпуса:  0.251025641025641
F1-score для корпуса:  0.21597759103641456


На пересечении

In [288]:
get_scores(df, 'intersection_keys', 'tf_idf')

Precision для корпуса:  0.19111111111111112
Recall для корпуса:  0.5204780900214336
F1-score для корпуса:  0.278980971571327


На объединении

In [289]:
get_scores(df, 'union_keys', 'tf_idf')

Precision для корпуса:  0.47999999999999987
Recall для корпуса:  0.2119550889217412
F1-score для корпуса:  0.29364487179508586


Лучший на пересечении - recall

Лучший F1-score и precision на объединении

**В итоге** лучшим вариантом мне кажется нужно считать не совсем те слова, которые мы выбрали в качестве эталонных, как показали результаты (говоря о точности) лучше себя показывает объединение слова, тогда у нас повышается результат на precision (много где). Если говорить о том, какой метод показывает себя лучше, то это - tf-idf (precision + объединение + без морф анализа) и tf-idf (precision + объединение + морф анализ). Самый хороший результат по описанному выше - precision + объединение + морф анализ

### 6: (1 балл) Описать ошибки автоматического выделения ключевых слов (что выделяется лишнее, что не выделяется); предложить свои методы решения этих проблем.



Ошибки автоматического выделения сильно зависят от того, какой методы мы используем.

Для **RAKE**:



*   выделяется достатотчно много конструкций, при этом, они кажутся мне достаточно оптимальными (но как показал результат, точность очень слабая). Некоторые слова, которые туда попопают, кажутся лишними, но и не совсем стоп-словами для того, чтобы их удалять. Например, "исходить", "связать", "далеко" и т.д. (очень много глаголов, которые частотны для типа статей, то же касается слов типа "эксперт", "автор", "специалист"), которые, кажется, недостаточно хорошо относятся к конкретному тексту и являются (хотя мне кажется, что нет), соответственно, ключевыми словами по мнению метода. Такие конструкции я отнесла бы к характеризующим любую статью вещам (любой текст), но не только в применении к самой конкретной статье. Поэтому тут я вижу достаточно большую ошибку в том, чтобы отбирать дествительно важное.


*  есть некоторая проблема, котрая связана с минимальной частотностью: если мы берем частотность 2 - работает не очень, видимо тут не хватает длины текстов (+ все тексты разные). Если же берем 1 то добавляется некоторый мусор (описанный выше), рейк перестает хорошо подбирать слова. (это было уже показано при использовании метода (6 слов на 2 и 138 слов на 1) ). Было бы идеально использовать 2 вместо 1, но, мне кажется, что тексты достаточно большие и в них просто не может быть всего 6 ключевых слов


Для **TextRank**:


*   не учитывает стоп-слова (для этого пришлось писать отдельную функцию)

*   все слова, где есть "й" переводятся в "и", таким образом слово рушится в своей изначальной форме

*   выделяет мало многословных конструкций (некоторые из них относятся к статье достаточно опосредственно, например, "эксперт утверждать"). А какие-то штуки типа "ядерная угроза" выделяются либо очень редко, либо вообще не выделяются. Отчасти повтораяется ошибка рейка о выделении просто общих слов для статей (это кажется лишним)


Для **TF-IDF**:


*   может получить только ключевые слова, без фраз (униграммы)

*   выбор можно улучшить путем добавления эмбеддингов в дальнейшем

*   но этот метод работает на самом деле лучше всего



---


Что выделяется лишнего и что не выделяется:


In [296]:
df

,texts,key_words,keys,length,lemm_texts,new_key_words,len_new_key_words,len_keys,intersection_keys,len_inters,union_keys,len_union,etalon_keys,len_etalon,rake,len_rake,text_rank,len_text_rank,tf_idf,len_tf_idf,etalon_n_grams,триграммы,биграммы,униграммы,rake_update,text_rank_update,tf_idf_update,rake_update_len,text_rank_update_len,tf_idf_update_len
0,"Эксперт утверждает, что Вашингтон может проигр...","{'вашингтон': ['NOUN', 2], 'россия': ['NOUN', ...","[вашингтон, россия, центр, c, csbs, цсбо, пент...",617,эксперт утверждать вашингтон мочь проиграть во...,"[вашингтон, россия, война, цсбо, 5 июль, 30 го...",56,54,"[рсмд, договор, цсбо, америка, ракета, твд, to...",17,"[рсмд, крылатый ракета, твд, 30 год, сухопутны...",93,"[рсмд, крылатый ракета, цсбо, ракета, америка,...",30,"[военно-промышленный комплекс, соглашение благ...",138,"[ракета, мочь, американскии, договор, такои, а...",44,"[ракета, россия, китай, сша, мочь, соединить, ...",45,"{'униграммы': ['NOUN_', 'ADV_', 'ENG_', 'ADP_'...","[война с россия, год специалист центр, крупный...","[с россия, в конец, конец май, год специалист,...","[эксперт, утверждать, вашингтон, мочь, проигра...","[военно-промышленный комплекс, призыв белый до...","[ракета, мочь, договор, америка, сша, россия, ...","[ракета, россия, китай, сша, мочь, соединить, ...",94,25,42
1,"Военный эксперт Антон Лавров — о том, почему ...","{'ал': ['NAME', 1], 'украина': ['NOUN', 8], 'ш...","[ал, украина, штурмовик, st, бразилия, самолет...",860,военный эксперт антон лавров почему приобретен...,"[эксперт, украина, штурмовик, бразилия, учебно...",50,65,"[ракета, ввс, зрк, штурмовик, истребитель, md-...",16,"[бомбовый нагрузка, летный характеристика, 30 ...",99,"[ракета, бомбовый нагрузка, летный характерист...",30,"[совершенный обзорно-прицельный станция, закуп...",183,"[самолет, год, боевои, сша, легкии, ввс этот с...",56,"[самолёт, год, tucano, боевой, украина, super,...",45,"{'униграммы': ['NOUN_', 'NUM_', 'ADP_', 'ADJ_'...","[эксперт антон лавров, приобретение украина шт...","[военный эксперт, эксперт антон, антон лавров,...","[военный, эксперт, антон, лавров, приобретение...","[совершенный обзорно-прицельный станция, ниша ...","[год, сша, применение, ракета, противник, маши...","[самолёт, год, tucano, боевой, украина, штурмо...",119,28,38
2,Военный эксперт спрогнозировал итоги войны СШ...,"{'военный': ['NOUN', 6], 'война': ['NOUN', 5],...","[военный, война, сша, россия, море, черный, лг...",622,военный эксперт спрогнозировать итог война сша...,"[военный, сша, россия, война, черный море, мес...",56,83,"[крым, военный, акватория, мина, marinetraffic...",19,"[крым, военный, национальный, шестой, противол...",119,"[крым, развертывание корабль, второй мировой в...",40,"[вице-адмирал лиза франчетти, нарушение россий...",120,"[море, воина сша, американскии авианосец, кора...",48,"[море, сша, чёрный, корабль, американский, вое...",45,"{'униграммы': ['NOUN_', 'ADJ_', 'ENG_'], 'бигр...","[итог война сша, россия на море, море чёрный м...","[военный эксперт, итог война, война сша, на мо...","[военный, эксперт, спрогнозировать, итог, войн...","[вице-адмирал лиза франчетти, нарушение россий...","[море, корабль вмс, флот, эсминец, проводить, ...","[море, сша, чёрный, корабль, американский, вое...",82,26,39
3,В составе бригад управления Минобороны РФ фор...,"{'управление': ['NOUN', 4], 'минобороны': ['NO...","[управление, минобороны, рф, мобильный, ретран...",717,состав бригада управление минобороны рф формир...,"[миноборона, рота, мобильный ретранслятор, дал...",61,52,"[техника, оборудование, вышка, сигнал, рф, кам...",13,"[военный, пострадать, спецблок, дистанционный ...",100,"[техника, аппаратура, армейский командир, дист...",40,"[помощь обычный интернет-мессенджер, антенный ...",147,"[сотовыи связь, новыи, зона, раион, военныи ил...",48,"[связь, сотовый, сеть, оператор, мобильный, мо...",45,"{'униграммы': ['NOUN_', 'ADP_', 'ADJ_'], 'бигр...","[состав бригада управление, бригада управление...","[в состав, состав бригад

**RAKE**

In [307]:
# что есть в объединении и нет в рейке
ter = ''

re = list(set(df['union_keys'].tolist()[0]) - set(df['rake_update'].tolist()[0]))

for el in re:
    ter += el + ' | '

In [308]:
ter

'рсмд | крылатый ракета | твд | 30 год | специалист | 100 единиц | базирование | сухопутный | сдерживание | москва | боеголовка | боеспособность | нво | радиус действия | 1990-х | ii | уолтон | наземный | обороноспособность | нато | статья | pershing | тимоти | нв | запрещенный | партнер америка | доклад | ту | 5 июль | фрнр | 30 лет | боевой столкновение | tnyt | противодействие | дальность | комплекс | американский | белый | лодка | оснащение | цсбо | америка | неядерный | tomahawk | белый дом | атаковать | t | война | пентагон | наземный базирование | марианский | средний | c | зд | соединенный штат | мо | вс | иванов | центр | отказываться | территория | 100 единица | сотрудник | вооружение | переговоры | планировка | 5 июля | средний дальность | перепрофилирование | 100 запрещенных | сш | поражение | '

In [309]:
# наоборот
ter_2 = ''

re_2 = list(set(df['rake_update'].tolist()[0]) - set(df['union_keys'].tolist()[0]))

for el in re_2:
    ter_2 += el + ' | '

In [310]:
ter_2

'ракета наземный базирование | объявить | закрытие дрсмд | привести | страна нато | интерес союзник | перепрофилирование существующий модель | иметь | тысяча ракета | разместить | название | связанный | заказ пентагон | цсбый | обычный боеголовка | обусловить | оснащение подобный ракета | путать | необходимость заключение дрсмд | состязание | появление | суббота | пятница | свидетельствовать | утверждение | ядерный боеголовка | связать | планировка игровой поле | вооружение тысяча | море | антагонист америка | необходимый дальность | неядерный боеголовка | агрессивный действие | северный марианский остров | заголовок | работать | подводный лодка | точный | пробел | число | призыв белый дом | прошлый неделя | восстановление ракета сша | короткий срок | одобрение | тревожный | подразделение россия | финансовый | настоящий война | сухопутный твд | составитель доклад | партнёр америка | игровой поле | американский территория | берег | walton | вс америка | конец май | американский дипломат

RAKE не учитывает всякие измерения и годы или очень редко выбирает аббревиатуры на английском (в объединении этого гораздо больше). Выделяет много дней недели (суббота/воскресень), всякие ненужные глаголы, существительные и прилагальные (работать, точный, число). При этом может выделять и разные хорошие штуки, которые не учитывает объединение (часть двустороннего догорва, ракета наземного базирования и т.д.). В целом среди мусора можно выделить то, что описано тут (скорее слова относятся к любой статье, поэтому они не ключевые (большинство глаголов при разности, прилагательных и существительных)).



**RankText**

In [311]:
# что есть в объединении и нет в ранктексте
ter_3 = ''

re_3 = list(set(df['union_keys'].tolist()[0]) - set(df['text_rank_update'].tolist()[0]))

for el in re_3:
    ter_3 += el + ' | '

In [312]:
ter_3

'рсмд | крылатый ракета | твд | 30 год | специалист | 100 единиц | базирование | сухопутный | корабль | сдерживание | москва | гуам | боеспособность | нво | радиус действия | 1990-х | впк | азия | ii | уолтон | наземный | обороноспособность | нато | аляска | pershing | тимоти | нв | запрещенный | партнер америка | доклад | ту | 5 июль | фрнр | 30 лет | боевой столкновение | tnyt | противодействие | китай | комплекс | американский | белый | лодка | вашингтон | оснащение | цсбо | база | неядерный | csbs | tomahawk | оперативно-тактический | белый дом | атаковать | t | война | наземный базирование | марианский | заключение договор | средний | c | зд | соединенный штат | мо | вс | иванов | центр | территория | ядерный | 100 единица | сотрудник | вооружение | переговоры | планировка | военно-промышленный комплекс | базироваться | 5 июля | средний дальность | перепрофилирование | 100 запрещенных | сш | поражение | '

In [313]:
# наоборот
ter_4 = ''

re_4 = list(set(df['text_rank_update'].tolist()[0]) - set(df['union_keys'].tolist()[0]))

for el in re_4:
    ter_4 += el + ' | '

In [314]:
ter_4

'оценка | игнорировать | мочь | время | оснастить | позволить | тысяча | запретить | вероятность | поле | модель | тимоти уолтон | '

RankText, как и рейк, не завтрагивает некоторые слова, которые были выделены вручную (многое, что связано с числами, например, английские аббревиатуры, и сокращения в основном). При этом, очень мало того, что есть в ранктекст, но нет в объединении: НЕР (отчасти) и ряд глаголов, но, их как-то сложно объединить в одну группу (в целом, они тоже могут относится к любой статье)

**Tf-idf**

In [315]:
# что есть в объединении и нет в ранктексте
ter_5 = ''

re_5 = list(set(df['union_keys'].tolist()[0]) - set(df['tf_idf_update'].tolist()[0]))

for el in re_5:
    ter_5 += el + ' | '

In [316]:
ter_5

'рсмд | крылатый ракета | твд | 30 год | специалист | 100 единиц | сухопутный | сдерживание | корабль | москва | боеспособность | гуам | нво | радиус действия | 1990-х | впк | азия | ii | обороноспособность | нато | аляска | статья | pershing | тимоти | нв | запрещенный | партнер америка | ту | 5 июль | фрнр | 30 лет | боевой столкновение | tnyt | противодействие | комплекс | белый | лодка | оснащение | цсбо | база | неядерный | csbs | tomahawk | оперативно-тактический | белый дом | атаковать | t | наземный базирование | марианский | заключение договор | c | зд | соединенный штат | мо | вс | иванов | центр | отказываться | территория | 100 единица | сотрудник | вооружение | переговоры | планировка | военно-промышленный комплекс | базироваться | 5 июля | средний дальность | перепрофилирование | 100 запрещенных | сш | поражение | '

In [317]:
# наоборот
ter_6 = ''

re_6 = list(set(df['tf_idf_update'].tolist()[0]) - set(df['union_keys'].tolist()[0]))

for el in re_6:
    ter_6 += el + ' | '

In [318]:
ter_6

'привести | разместить | действие | войско | соединить | поле | эксперт | обычный | позволить | вероятность | сила | оснастить | год | проиграть | мочь | соблюдать | призвать | штат | подразделение | отметить | запретить | '

Не учитывает часть тех штук, где есть количество, не учитывает биграммы и триграммы. В остальном, у него есть слова, которые кажутся очень важными (войско, штат), при этом выделяет важные глаголы (резместить, отменить, запретить). И вообще работает лучше всего